# ANC Databricks: Wikipedia + FAISS + Databricks + StateGraph

This notebook combines two Databricks workflows in one place:

- Baseline notebook pipeline: Wikipedia -> chunking -> FAISS -> Databricks-hosted Q&A with `ChatDatabricks`
- Canonical orchestration diagram + run flow via shared `naturalist_companion.stategraph_shared`

Provider mapping used here:
- `ChatOllama` -> `ChatDatabricks`
- `OllamaEmbeddings` -> `DatabricksEmbeddings`
- Same `WikipediaLoader` + `FAISS` retrieval flow


## Prerequisites (Run First)

### Databricks Free Edition setup

- Create/sign in to your Databricks Free Edition workspace.
- Create a Personal Access Token (PAT) in Databricks.
- For local notebook runs, set:
  - `export DATABRICKS_HOST="https://<your-workspace-host>"`
  - `export DATABRICKS_TOKEN="<your-pat>"`
- Ensure your user/token has `Can Query` permission for the selected model endpoints.

### Foundation model endpoints

This notebook defaults to Databricks-hosted Foundation Model API endpoints:
- Embeddings: `databricks-bge-large-en`
- Chat: `databricks-meta-llama-3-1-8b-instruct`

If your workspace exposes different endpoint names, override with:
- `DATABRICKS_EMBEDDING_ENDPOINT`
- `DATABRICKS_LLM_ENDPOINT`

### Python dependency notes

- Run the next code cell in a fresh kernel to install notebook dependencies.
- `requirements` files are optional. If you imported this notebook directly in Databricks, use the direct package install option in the next cell.
- If you hit `TqdmWarning: IProgress not found`, install `ipywidgets jupyterlab_widgets` and restart the kernel.
- If you hit macOS OpenMP kernel crashes, launch Jupyter with `KMP_DUPLICATE_LIB_OK=TRUE`.

### IDE notebook stability tips

- Ensure the notebook kernel points to this project interpreter: `/Users/ryan/Developer/naturalist-companion/.venv/bin/python`
- After dependency changes, use kernel restart then run all cells from top.
- Keep one notebook kernel active at a time while testing remote model calls.


In [ ]:
# Optional dependency installs (choose one path).

# Local repo checkout path (uses project requirements file):
# %pip install -q -r ../requirements-dbrx-dev.txt

# Databricks browser-import path (no local requirements file needed):
# %pip install -q -U backoff databricks-langchain "mlflow-skinny[databricks]" langchain==0.3.27 langchain-core==0.3.83 langchain-community==0.3.31 langchain-text-splitters==0.3.11 faiss-cpu==1.13.2 wikipedia==1.4.0 ipywidgets jupyterlab_widgets

# In Databricks notebooks, restart Python after %pip installs:
# dbutils.library.restartPython()


In [ ]:
#######################################################################################################

###### Python Package Imports for this notebook                                                  ######

#######################################################################################################


import os
import warnings
from threading import Event, Thread

# Mitigate common macOS OpenMP duplicate-library crashes in notebook kernels.
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")

# Silence noisy tqdm widget warning in IDE notebooks when rich progress widgets are unavailable.
warnings.filterwarnings("ignore", message=".*IProgress not found.*")


# LangChain moved WikipediaLoader in newer releases; keep backward compatibility.
try:
    from langchain_community.document_loaders import WikipediaLoader
except ImportError:
    from langchain.document_loaders import WikipediaLoader

try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from databricks_langchain import ChatDatabricks, DatabricksEmbeddings


def _start_heartbeat(label: str, every_s: float = 8.0):
    stop = Event()

    def _run():
        elapsed = 0.0
        while not stop.wait(every_s):
            elapsed += every_s
            print(f"[{label}] still running... {elapsed:.0f}s elapsed")

    thread = Thread(target=_run, daemon=True)
    thread.start()
    return stop


def _show_databricks_auth_status() -> None:
    host = os.environ.get("DATABRICKS_HOST", "").strip()
    has_token = bool(os.environ.get("DATABRICKS_TOKEN", "").strip())
    in_runtime = bool(os.environ.get("DATABRICKS_RUNTIME_VERSION", "").strip())

    if in_runtime:
        print("[env] Databricks runtime detected; workspace auth should be available via runtime context.")
        return

    if host and has_token:
        print(f"[env] Databricks auth env detected: DATABRICKS_HOST={host}")
    else:
        print("[env] Databricks auth env not fully set. For local runs set DATABRICKS_HOST and DATABRICKS_TOKEN.")


try:
    import ipywidgets as _ipywidgets
    print(f"[env] ipywidgets detected: v{_ipywidgets.__version__}")
except Exception:
    print("[env] ipywidgets not found in this kernel. Run `%pip install -q ipywidgets` and restart the kernel.")

def _raise_databricks_hint(stage: str, endpoint: str, exc: Exception) -> None:
    message = str(exc)
    hints = [
        f"[{stage}] Databricks call failed for endpoint={endpoint!r}.",
        f"Underlying error: {message}",
    ]

    lower_msg = message.lower()
    if "invalid access token" in lower_msg or "403" in lower_msg:
        hints.append("Check that DATABRICKS_TOKEN is valid and matches DATABRICKS_HOST.")
        hints.append("In Free Edition, regenerate a PAT and retry if the old token is expired.")
    if "resource_does_not_exist" in lower_msg or "404" in lower_msg:
        hints.append("Verify the endpoint name in Databricks Serving and update notebook env overrides.")
    if "permission" in lower_msg or "can query" in lower_msg:
        hints.append("Ensure your user/token has Can Query permission on the endpoint.")

    raise RuntimeError("\n".join(hints)) from exc


_show_databricks_auth_status()


#######################################################################################################

###### Config (Define LLMs, Embeddings, Vector Store, Data Loader specs)                         ######

#######################################################################################################


# DataLoader Config
query_terms = [
    "roadcut",
    "geology",
    "sedimentary rock",
    "stratigraphy",
]
max_docs = 3  # Fast local iteration setting.

# Stage 2 chunking + batching controls (keep small for interactive runs).
chunk_size = 1200
chunk_overlap = 150
embedding_batch_size = 8


# Retriever Config
k = 1
EMBEDDING_MODEL_ENDPOINT = os.environ.get("DATABRICKS_EMBEDDING_ENDPOINT", "databricks-bge-large-en")


# LLM Config
LLM_ENDPOINT_NAME = os.environ.get("DATABRICKS_LLM_ENDPOINT", "databricks-meta-llama-3-1-8b-instruct")
TEMPERATURE = 0.0


# Response style controls (Roadside Geology audience: curious drivers, practical field learners).
RESPONSE_TONE = "field-guide"
MAX_BULLETS_PER_SECTION = 4


## Query Prompt (Edit This Cell)

Use the next code cell to set the active question(s).

Question types this notebook is designed for:
- Detour geology: legal pull-offs or short walks near a route segment
- Safety-first prompts: where to stop and what to avoid roadside
- Route constraints: city/exit anchors plus max detour minutes
- Beginner field interpretation: what visual clues to look for and why they matter

Tip: Include your nearest city or exit and your max detour time to improve stop recommendations.


In [ ]:
example_question = "I am on I-81 near Hagerstown with a 30-minute detour. Where can I safely stop to observe folded Valley-and-Ridge strata, and what exactly should I look for?"

example_questions = [
    "I am driving I-81 near Bristol, TN. Give me two legal pull-off stops where I can see clear sedimentary layering, and tell me exactly what to look for.",
    "Near I-81 between Winchester and Strasburg, where can I safely stop to see Valley-and-Ridge structure, and what field clues confirm folding?",
    "I have 45 minutes near Hagerstown, MD. What roadside geology stop gives the best payoff for a beginner, and what story does the outcrop tell?",
    "Along I-81 in the Shenandoah Valley, point me to a short-walk stop to compare rock type and landform, then explain why that match matters.",
    "On an I-81 drive day, suggest one stop where I can observe evidence of ancient seas or sediment transport, with specific visual clues.",
]

# StateGraph run can use the same prompt by default; edit independently if desired.
stategraph_question = example_question


In [ ]:
#######################################################################################################

###### Stage 1/3: Wikipedia Data Load                                                            ######

#######################################################################################################


from time import perf_counter

print("[stage 1/3] Starting Wikipedia document load...")
query = " ".join(query_terms) if isinstance(query_terms, list) else query_terms
print(f"[stage 1/3] query={query!r}, max_docs={max_docs}")

heartbeat = _start_heartbeat("stage 1/3 wikipedia load", every_s=8.0)
t0 = perf_counter()
try:
    docs = WikipediaLoader(query=query, load_max_docs=max_docs).load()
finally:
    heartbeat.set()
t1 = perf_counter()

print(f"[stage 1/3] Loaded {len(docs)} document(s) in {t1 - t0:.2f}s")
if not docs:
    raise RuntimeError("No documents loaded from Wikipedia. Adjust query_terms/max_docs and re-run stage 1.")

print("[stage 1/3] Sample titles:")
for i, doc in enumerate(docs[:3], start=1):
    title = str((doc.metadata or {}).get("title") or f"doc_{i}")
    source = str((doc.metadata or {}).get("source") or "n/a")
    print(f"  {i}. {title} ({source})")


In [ ]:
#######################################################################################################

###### Stage 2/3: Build + Save FAISS Index, Then Retrieve                                        ######

#######################################################################################################


import os
from pathlib import Path
from time import perf_counter

if "docs" not in globals() or not docs:
    raise RuntimeError("`docs` not found. Run Stage 1/3 first.")

print(f"[stage 2/3] Building embeddings with endpoint={EMBEDDING_MODEL_ENDPOINT!r}...")
print(
    f"[stage 2/3] Chunking docs with chunk_size={chunk_size}, chunk_overlap={chunk_overlap}, "
    f"embedding_batch_size={embedding_batch_size}"
)

splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
split_docs = splitter.split_documents(docs)
if not split_docs:
    raise RuntimeError("Chunking produced 0 documents. Adjust chunk_size/chunk_overlap and retry.")

total_chars = sum(len(str(d.page_content or "")) for d in split_docs)
print(f"[stage 2/3] Prepared {len(split_docs)} chunk(s), total_chars={total_chars}")

embeddings = DatabricksEmbeddings(endpoint=EMBEDDING_MODEL_ENDPOINT)

batch_size = max(1, int(embedding_batch_size))
vector_store = None

heartbeat = _start_heartbeat("stage 2/3 embedding/index", every_s=8.0)
t2 = perf_counter()
try:
    try:
        for start in range(0, len(split_docs), batch_size):
            batch = split_docs[start : start + batch_size]
            b0 = perf_counter()
            if vector_store is None:
                vector_store = FAISS.from_documents(batch, embeddings)
            else:
                vector_store.add_documents(batch)
            b1 = perf_counter()

            done = min(start + batch_size, len(split_docs))
            pct = (100.0 * done) / len(split_docs)
            print(
                f"[stage 2/3] Embedded batch {start // batch_size + 1}: "
                f"{done}/{len(split_docs)} chunks ({pct:.1f}%) in {b1 - b0:.2f}s"
            )
    except Exception as exc:
        _raise_databricks_hint("stage 2/3 embedding/index", EMBEDDING_MODEL_ENDPOINT, exc)
finally:
    heartbeat.set()

t3 = perf_counter()
if vector_store is None:
    raise RuntimeError("Vector store was not created.")

print(f"[stage 2/3] Built FAISS index in {t3 - t2:.2f}s")


faiss_base = os.environ.get("ANC_FAISS_DIR", "").strip()
if faiss_base:
    faiss_dir = (Path(faiss_base).expanduser() / "anc_dbrx").resolve()
else:
    faiss_dir = (Path.home() / "DATA" / "naturalist-companion" / "faiss" / "anc_dbrx").resolve()

faiss_dir.mkdir(parents=True, exist_ok=True)
vector_store.save_local(str(faiss_dir))
print(f"[stage 2/3] Saved FAISS index to: {faiss_dir}")


print(f"[stage 2/3] Running similarity search for question={example_question!r}, k={k}...")
results = vector_store.similarity_search(example_question, k=k)
print(f"[stage 2/3] Retrieved {len(results)} result(s)")

for i, res in enumerate(results, start=1):
    title = str((res.metadata or {}).get("title") or f"result_{i}")
    source = str((res.metadata or {}).get("source") or "n/a")
    snippet = str(res.page_content or "")[:220].replace("\n", " ")
    print(f"  {i}. {title} ({source})")
    print(f"     {snippet}...")


In [ ]:
#######################################################################################################

###### Stage 3/3: Generate Answer with ChatDatabricks                                            ######

#######################################################################################################


from time import perf_counter

if "vector_store" not in globals():
    raise RuntimeError("`vector_store` not found. Run Stage 2/3 first.")

print(f"[stage 3/3] Generating answer with endpoint={LLM_ENDPOINT_NAME!r}...")
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=TEMPERATURE)

voice_instructions = f"""
You are writing in a concise Roadside Geology field-guide voice for curious drivers.
Tone:
- Plainspoken, observant, and practical (not academic).
- Emphasize what can be seen from legal/safe pull-offs or short walks.
- Explain key geology in everyday language, then add one precise term when useful.
- Include safety and access realism (do not suggest unsafe roadside behavior).
Output format:
1) "Where to stop" (up to {MAX_BULLETS_PER_SECTION} bullets)
2) "What to look for" (up to {MAX_BULLETS_PER_SECTION} bullets)
3) "Why it matters" (2-4 sentences)
4) "Citations" (Wikipedia URLs only)
""".strip()


def _context_for_question(question: str, top_k: int = 2) -> str:
    local_results = vector_store.similarity_search(question, k=max(1, top_k))
    lines = []
    for i, res in enumerate(local_results, start=1):
        title = str((res.metadata or {}).get("title") or f"result_{i}")
        source = str((res.metadata or {}).get("source") or "n/a")
        snippet = str(res.page_content or "")[:450].replace("\n", " ")
        lines.append(f"[{i}] {title} ({source}) :: {snippet}")
    return "\n".join(lines)


def answer_question(question: str) -> str:
    context_block = _context_for_question(question, top_k=max(1, k))
    prompt = (
        f"Use only the provided Wikipedia-grounded context when you can.\n\n"
        f"Question: {question}\n\n"
        f"Context:\n{context_block}\n\n"
        f"Style requirements:\n{voice_instructions}"
    )

    heartbeat = _start_heartbeat("stage 3/3 llm", every_s=8.0)
    t0 = perf_counter()
    try:
        try:
            response = llm.invoke(prompt)
        except Exception as exc:
            _raise_databricks_hint("stage 3/3 llm", LLM_ENDPOINT_NAME, exc)
    finally:
        heartbeat.set()
    dt = perf_counter() - t0
    print(f"[stage 3/3] LLM response received in {dt:.2f}s")
    return str(response.content)


print(f"[stage 3/3] Primary question:\n- {example_question}")
primary_answer = answer_question(example_question)
print("\nAnswer:\n")
print(primary_answer)


In [ ]:
#######################################################################################################

###### Stage 3b/3: Run All Example Questions                                                     ######

#######################################################################################################


if "answer_question" not in globals():
    raise RuntimeError("`answer_question` not found. Run Stage 3/3 first.")

if "example_questions" not in globals() or not example_questions:
    raise RuntimeError("`example_questions` is empty. Check config cell.")

all_answers = []
print(f"[stage 3b/3] Running {len(example_questions)} example question(s)...")

for i, q in enumerate(example_questions, start=1):
    print("\n" + "=" * 110)
    print(f"[stage 3b/3] Question {i}/{len(example_questions)}")
    print(q)
    print("=" * 110)

    answer = answer_question(q)
    all_answers.append({"question": q, "answer": answer})

    print("\nResponse:\n")
    print(answer)

print(f"\n[stage 3b/3] Completed {len(all_answers)} question(s).")


## Canonical Workflow Diagram (StateGraph)

StateGraph is the canonical workflow diagram for this notebook because it reflects the shared orchestration logic and is less likely to drift than a separate static diagram.


In [ ]:
# Dependencies should already be installed from the setup cell above.

from pathlib import Path
import os
import sys

from IPython.display import Image, Markdown, display


def _candidate_src_paths() -> list[Path]:
    candidates: list[Path] = []

    # Optional explicit override.
    env_src = os.environ.get("NATURALIST_COMPANION_SRC", "").strip()
    if env_src:
        candidates.append(Path(env_src))

    # If running inside Databricks, derive repo path from notebook context.
    try:
        notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        # Example notebook_path: /Repos/user/repo/notebooks/anc_dbrx
        if "/notebooks/" in notebook_path:
            repo_workspace_path = "/Workspace" + notebook_path.split("/notebooks/", 1)[0]
            candidates.append(Path(repo_workspace_path) / "src")
    except Exception:
        pass

    # Relative paths (works in local and some Databricks repo executions).
    cwd = Path.cwd()
    candidates.extend([
        cwd / "src",
        cwd.parent / "src",
        cwd.parent.parent / "src",
    ])

    # Databricks Repos filesystem fallback scan.
    repos_root = Path("/Workspace/Repos")
    if repos_root.exists():
        for pkg_dir in repos_root.glob("*/*/src/naturalist_companion"):
            candidates.append(pkg_dir.parent)

    # De-duplicate while preserving order.
    deduped: list[Path] = []
    seen: set[str] = set()
    for item in candidates:
        key = str(item)
        if key in seen:
            continue
        seen.add(key)
        deduped.append(item)
    return deduped


STATEGRAPH_AVAILABLE = False
STATEGRAPH_SRC_PATH = None
_stategraph_import_error = None

for src_path in _candidate_src_paths():
    if not (src_path / "naturalist_companion").exists():
        continue
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    STATEGRAPH_SRC_PATH = src_path
    break

try:
    from naturalist_companion.stategraph_shared import (
        build_stategraph_app,
        run_i81_eval_harness,
        run_stategraph,
    )
    STATEGRAPH_AVAILABLE = True
    if STATEGRAPH_SRC_PATH is not None:
        print(f"[stategraph] Loaded naturalist_companion from: {STATEGRAPH_SRC_PATH}")
    else:
        print("[stategraph] Loaded naturalist_companion from current Python path")
except Exception as exc:
    _stategraph_import_error = exc
    STATEGRAPH_AVAILABLE = False
    display(
        Markdown(
            "**StateGraph module is unavailable in this workspace.**\n\n"
            "Baseline Databricks stages (Wikipedia -> FAISS -> ChatDatabricks) still run.\n\n"
            "If this repo is synced in Databricks Repos, open the notebook from the Repo path "
            "(not a copied workspace file), or set `NATURALIST_COMPANION_SRC` to your repo `src` path."
        )
    )
    print(f"[stategraph] import error: {type(exc).__name__}: {exc}")


In [ ]:
if not STATEGRAPH_AVAILABLE:
    print("[stategraph] Skipping canonical diagram: naturalist_companion not available.")
else:
    provider = 'databricks'
    app = build_stategraph_app(provider=provider)
    print('Compiled StateGraph successfully for provider:', provider)

    # Render a real image (PNG bytes) instead of plain Mermaid text.
    try:
        png_bytes = app.get_graph().draw_mermaid_png()
        display(Image(data=png_bytes))
    except Exception as exc:
        display(Markdown(f'Graph render fallback (text). Error: `{type(exc).__name__}: {exc}`'))
        print(app.get_graph().draw_mermaid())


In [ ]:
if not STATEGRAPH_AVAILABLE:
    print("[stategraph] Skipping run_stategraph: naturalist_companion not available.")
else:
    result = run_stategraph(
        stategraph_question,
        provider='databricks',
        config={'artifact_root': 'out/stategraph/notebook_runs', 'max_retrieval_attempts': 3, 'citation_coverage_threshold': 0.80},
    )
    final_output = result['final_output']
    print('Question:', stategraph_question)
    print('Provider:', final_output['provider'])
    print('Route:', final_output['route_decision']['decision'])
    print('Quality passed:', final_output['quality']['passed'])
    print('Attempts:', final_output['retrieval_attempts'])
    print('Artifact dir:', result['artifact_dir'])
    print('Response:')
    print(final_output['answer']['response'])


In [ ]:
if not STATEGRAPH_AVAILABLE:
    print("[stategraph] Skipping eval harness: naturalist_companion not available.")
else:
    report = run_i81_eval_harness(
        provider='databricks',
        config={'artifact_root': 'out/stategraph/notebook_eval', 'max_retrieval_attempts': 3, 'citation_coverage_threshold': 0.80},
    )
    print(report['summary'])
    print(report['artifact_root'])
